In [1]:
rm -rf Django

# create workspace
mkdir Django
cd Django

In [2]:
# create project
django-admin startproject mysite
cd mysite

In [3]:
# create webapp
python manage.py startapp polls

In [4]:
# create new view
cat << EOF > polls/views.py
from django.http import HttpResponse


def index(request):
    return HttpResponse("Hello, world. You're at the polls index.")
EOF

In [5]:
# add url pattern to point at a new view
cat << EOF > polls/urls.py
from django.urls import path

from . import views

urlpatterns = [
    path('', views.index, name='index'),
]
EOF

In [6]:
# add a link to view in our webapp

sed -i -e "/urlpatterns/ a \    path('polls/', include('polls.urls'))," mysite/urls.py
sed -i -e "/from[ ]django/ s/path/include, path/" mysite/urls.py

In [7]:
# run the server
python manage.py runserver 7000 &

[1] 8969


In [8]:
firefox http:localhost:7000/polls

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).

You have 18 unapplied migration(s). Your project may not work properly until you apply the migrations for app(s): admin, auth, contenttypes, sessions.
Run 'python manage.py migrate' to apply them.
June 05, 2021 - 19:05:52
Django version 3.2.4, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.
firefox http:localhost:7000/polls^J[05/Jun/2021 19:06:02] "GET /polls HTTP/1.1" 301 0
[05/Jun/2021 19:06:02] "GET /polls/ HTTP/1.1" 200 40


In [ ]:
cat << EOF > polls/models.py
from django.db import models


class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')


class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)
EOF

In [ ]:
# Some of these applications make use of at least one database table, though, so we need to create the tables in the database before we can use them. To do that, run the following command:
python manage.py migrate

In [ ]:
sed -i -e "/INSTALLED_APPS/ a \    'polls.apps.PollsConfig'," mysite/settings.py
cat mysite/settings.py

In [ ]:
python manage.py makemigrations polls

In [ ]:
python manage.py sqlmigrate polls 0001

In [ ]:
python manage.py migrate

In [ ]:
# add additional views

In [ ]:
cat << EOF >> polls/views.py
def detail(request, question_id):
    return HttpResponse("You're looking at question %s." % question_id)

def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)

def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)
EOF

In [ ]:
cat << EOF > polls/urls.py
from django.urls import path

from . import views

urlpatterns = [
    # ex: /polls/
    path('', views.index, name='index'),
    # ex: /polls/5/
    path('<int:question_id>/', views.detail, name='detail'),
    # ex: /polls/5/results/
    path('<int:question_id>/results/', views.results, name='results'),
    # ex: /polls/5/vote/
    path('<int:question_id>/vote/', views.vote, name='vote'),
]
EOF

In [ ]:
firefox http://localhost:8000/polls/34/

In [ ]:
# firefox http://localhost:8000/polls/34/results

In [ ]:
# firefox http://localhost:8000/polls/34/vote

In [ ]:
cat << EOF > polls/views.py
from django.http import HttpResponse

def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    output = ', '.join([q.question_text for q in latest_question_list])
    return HttpResponse(output)
    
def detail(request, question_id):
    return HttpResponse("You're looking at question %s." % question_id)

def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)

def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)
EOF

In [ ]:
mkdir -p polls/templates/polls

In [ ]:
cat << EOF > polls/templates/polls/index.html
{% if latest_question_list %}
    <ul>
    {% for question in latest_question_list %}
        <li><a href="/polls/{{ question.id }}/">{{ question.question_text }}</a></li>
    {% endfor %}
    </ul>
{% else %}
    <p>No polls are available.</p>
{% endif %}
EOF

In [ ]:
cat << EOF > polls/views.py
from django.http import HttpResponse
from django.template import loader

from .models import Question


def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    template = loader.get_template('polls/index.html')
    context = {
        'latest_question_list': latest_question_list,
    }
    return HttpResponse(template.render(context, request))
EOF

In [ ]:
firefox http://localhost:8000/polls

In [ ]:
# launch server
python manage.py runserver
